In [1]:
import h5py
import numpy as np
import pyproj
import contextlib
from pyproj import CRS, Transformer
from datetime import datetime
import gzip
from pathlib import Path
import xarray as xr
import geopandas as gpd
import polars as pl

import sys
from pathlib import Path

%reload_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append("..")
sys.path.append("../..")


from database import get_engine
from utils.config_utils import load_config
from utils import plot_utils
from utils import graph_utils

PROJ4STR = "+proj=somerc +lat_0=46.95240555555556 +lon_0=7.439583333333333 +k_0=1 +x_0=2600000 +y_0=1200000 +ellps=bessel +towgs84=674.374,15.056,405.346,0,0,0,0 +units=m +no_defs"

In [2]:
radars_ch = gpd.read_file("/data/jenna/shapefiles/swiss_radars/swiss_radars.shp")
radars_ch = radars_ch.to_crs(PROJ4STR).drop_duplicates()

radars_ch

,Number,Country,Country ID,Old Countr,WMO Code,ODIM code,Location,Opera Stat,Latitude (,Longitude,...,Start Year,Height Ant,Diametr An,Beam (º),Gain (dB),Frequency,Single_RRR,Composite_,WRWP,geometry
0,1249,Switzerland,LSSW41,SW41,6661,chalb,Albis,1,47.2843,8.5120,...,1994,938,4.3,1,45,5.450,N,Y,Y,POINT Z (2681201.904 1237600.138 938)
1,1250,Switzerland,LSSW42,SW42,6699,chdol,La Dole,1,46.4251,6.0994,...,1995,1682,4.3,1,45,5.430,N,Y,Y,POINT Z (2497055.153 1142408.432 1682)
2,1251,Switzerland,LSSW43,SW43,6768,chlem,Monte Lema,1,46.0408,8.8332,...,1993,1626,4.3,1,45,5.455,N,Y,Y,POINT Z (2707955.819 1099765.931 1626)
3,1361,Switzerland,LSSW51,SW51,6726,chppm,Plaine Monte,1,46.3706,7.4866,...,2013,2937,4.3,1,45,5.468,N,Y,Y,POINT Z (2603690.248 1135470.494 2937)
4,1372,Switzerland,LSSW52,SW52,6776,chwei,Weissfluhgiptel,1,46.8350,9.7945,...,2015,2850,4.3,1,45,5.433,N,Y,Y,POINT Z (2779703.204 1189792.788 2850)


In [19]:
radars_ch[["ODIM code", "geometry"]]

,ODIM code,geometry
0,chalb,POINT Z (2681201.904 1237600.138 938)
1,chdol,POINT Z (2497055.153 1142408.432 1682)
2,chlem,POINT Z (2707955.819 1099765.931 1626)
3,chppm,POINT Z (2603690.248 1135470.494 2937)
4,chwei,POINT Z (2779703.204 1189792.788 2850)


In [25]:
radars_ch.iloc[0].geometry.x

2681201.9044294925

In [34]:
queries = []

for i, row in radars_ch.iterrows():

    sql = f"""

    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
        method,
        'dist_from_radar_{row['ODIM code']}' as quantity,
        'min' as statistic,
        ST_Distance(ST_POINT({row['geometry'].x}, {row['geometry'].y}), geometry) / 1e3 as value

    FROM raincells.stormcells

    ON CONFLICT DO NOTHING;
    """

    queries.append(sql)
    print(sql)



    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
        method,
        'dist_from_radar_chalb' as quantity,
        'min' as statistic,
        ST_Distance(ST_POINT(2681201.9044294925, 1237600.1383876249), geometry) / 1e3 as value

    FROM raincells.stormcells

    ON CONFLICT DO NOTHING;
    


    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
        method,
        'dist_from_radar_chdol' as quantity,
        'min' as statistic,
        ST_Distance(ST_POINT(2497055.1533623612, 1142408.4317818207), geometry) / 1e3 as value

    FROM raincells.stormcells

    ON CONFLICT DO NOTHING;
    


    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
    

In [35]:
from sqlalchemy.orm import Session
from sqlalchemy.sql import text

dbconf = {
    "host": "athras.fmi.fi",
    "port": 5432,
    "database": "raincells_swiss_data_v20250508",
    "schema": "raincells",
    "table": "stormcells",
    "user": "ritvanen",
    "password": "BorneArgon",
}
engine = get_engine(dbconf)

with Session(engine) as session:
    for query in queries:
        print(query)
        session.execute(text(query))
        session.commit()
        print("Query executed and committed.")



    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
        method,
        'dist_from_radar_chalb' as quantity,
        'min' as statistic,
        ST_Distance(ST_POINT(2681201.9044294925, 1237600.1383876249), geometry) / 1e3 as value

    FROM raincells.stormcells

    ON CONFLICT DO NOTHING;
    
Query executed and committed.


    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT
        "timestamp",
        identifier,
        method,
        'dist_from_radar_chdol' as quantity,
        'min' as statistic,
        ST_Distance(ST_POINT(2497055.1533623612, 1142408.4317818207), geometry) / 1e3 as value

    FROM raincells.stormcells

    ON CONFLICT DO NOTHING;
    
Query executed and committed.


    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, valu

In [ ]:
# Get smallest distance from any radar for each stormcell

from sqlalchemy.orm import Session
from sqlalchemy.sql import text

query = """

    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT distinct on ("timestamp", identifier, method) "timestamp", identifier, method, 'dist_from_radars', 'min', value
	FROM raincells.stormcell_rasterstats

    where quantity like 'dist_from_radar_%'

    order by "timestamp", identifier, method, value

    ON CONFLICT DO NOTHING;

"""

dbconf = {
    "host": "athras.fmi.fi",
    "port": 5432,
    "database": "raincells_swiss_data_v20250508",
    "schema": "raincells",
    "table": "stormcells",
    "user": "ritvanen",
    "password": "BorneArgon",
}
engine = get_engine(dbconf)

with Session(engine) as session:
    print(query)
    session.execute(text(query))
    session.commit()
    print("Query executed and committed.")



    INSERT INTO raincells.stormcell_rasterstats(
	"timestamp", identifier, method, quantity, statistic, value)

    SELECT distinct on ("timestamp", identifier, method) "timestamp", identifier, method, 'dist_from_radars', 'min', value
	FROM raincells.stormcell_rasterstats

    where quantity like 'dist_from_radar_%'

    order by "timestamp", identifier, method, value

    ON CONFLICT DO NOTHING;


Query executed and committed.


: 